In [59]:
import numpy as np
# import tensorflow as tf
from tensorflow.keras.datasets import imdb
from tensorflow.keras.datasets import reuters
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras.preprocessing import sequence
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt 
import pickle 

Loading the dataset

In [49]:
ruta = './datasets/'
df_0 = pd.read_csv(ruta+'simpsons/simpsons_dataset.csv').dropna()
target_simpsons = [
    'Lisa Simpson',
    'Bart Simpson',
    'Homer Simpson',
    'Marge Simpson',
    'Maggie Simpson',
    'Grampa Simpson'
]
data = df_0.loc[df_0['raw_character_text'].isin(target_simpsons)]
data.head(5)

,raw_character_text,spoken_words
1,Lisa Simpson,Where's Mr. Bergstrom?
3,Lisa Simpson,That life is worth living.
7,Bart Simpson,Victory party under the slide!
9,Lisa Simpson,Mr. Bergstrom! Mr. Bergstrom!
11,Lisa Simpson,Do you know where I could find him?


In [54]:
sentences = data['spoken_words'].values
y = data['raw_character_text'].values

Splitting the dataset

In [57]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.5, random_state=1000)

Vectorize the dataset

In [63]:
top_words = 5000
vectorize_layer = TextVectorization(
    ngrams=None, max_tokens=top_words, vocabulary=None,
    output_mode='int', output_sequence_length=None, pad_to_max_tokens=True,
)

vectorize_layer.adapt(sentences_train)
print(len(vectorize_layer.get_vocabulary()))

5000


In [64]:
x_train = vectorize_layer(np.array([[s] for s in sentences_train])).numpy()
x_test = vectorize_layer(np.array([[s] for s in sentences_test])).numpy()

y_train = np.array(y_train)
y_test = np.array(y_test)

In [ ]:
x_train

In [33]:
# (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=top_words)
# Parece que te tira el dataset como 25 mil train, 25 mil test, con cada entrada el embedding 'int' del doc.
# Esto nos tocaría usar un vectorizze layer sobre nuestros docs con output mode 'int'


In [65]:
# len(X_train[3])
print(type(x_train))
len(x_test)
print(len(y_test))

<class 'numpy.ndarray'>
33293


Pad or truncate all sequences to half of max lenght

In [66]:
# Podriamos usar esto o simplemente definir un número 'bueno' y truncar o paddear todo a ese número
# Con twitter depronto sí sirve un número de 25 por ejemplo.
max_doc_length = np.max([len(doc) for doc in x_train])

print('Máximo tamaño de documento en train:', max_doc_length)
x_train_padded = sequence.pad_sequences(x_train, maxlen=(max_doc_length))
x_test_padded = sequence.pad_sequences(x_test, maxlen=(max_doc_length))

Máximo tamaño de documento en train: 122


Enconding classification classes

In [67]:
encoder = LabelBinarizer()

encoder.fit(y_train)
y_train_bin = encoder.transform(y_train)
y_test_bin = encoder.transform(y_test)

In [68]:
num_classes = len(encoder.classes_)
print('Número de clases:', num_classes)

Número de clases: 6


## Model definition

In [73]:
# Acá hay que tener cuidado porque nuestro problema de clasificación No es binario
# Con los datos reales el Compile debe ser con 'categorical_crossentropy'
# Además el tamaño de la última softmax debe ser el mismo que el número de categorias
# Se tiene también que pasar los datos en Y a oneHotvectors, ese código lo podemos sacar del notebool model_V1

# create the model
embedding_vecor_length = 256
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_doc_length))
model.add(LSTM(100))
model.add(Dense(num_classes, activation='softmax'))

Compilation

In [74]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 122, 256)          1280000   
_________________________________________________________________
lstm_4 (LSTM)                (None, 100)               142800    
_________________________________________________________________
dense_4 (Dense)              (None, 6)                 606       
Total params: 1,423,406
Trainable params: 1,423,406
Non-trainable params: 0
_________________________________________________________________
None


Training

In [75]:
model.fit(x_train_padded, y_train_bin, validation_data=(x_test_padded, y_test_bin), epochs=5, batch_size=64)

Epoch 1/5
521/521 [==============================] - 21s 37ms/step - loss: 1.4290 - accuracy: 0.4115 - val_loss: 1.4005 - val_accuracy: 0.4194
Epoch 2/5
521/521 [==============================] - 20s 37ms/step - loss: 1.4064 - accuracy: 0.4135 - val_loss: 1.3983 - val_accuracy: 0.4194
Epoch 3/5
521/521 [==============================] - 20s 38ms/step - loss: 1.4040 - accuracy: 0.4144 - val_loss: 1.3983 - val_accuracy: 0.4194
Epoch 4/5
521/521 [==============================] - 25s 47ms/step - loss: 1.3968 - accuracy: 0.4209 - val_loss: 1.4000 - val_accuracy: 0.4194
Epoch 5/5
521/521 [==============================] - 36s 69ms/step - loss: 1.4053 - accuracy: 0.4132 - val_loss: 1.3991 - val_accuracy: 0.4194
